In [11]:
# Dataset Collection
import cv2
import numpy as np

k=0
cap=cv2.VideoCapture(0)

#Put these numbers for adding to the training dataset: 0 for rock,1 for paper,2 for scissors

while True:
    _,frame=cap.read()
    cv2.rectangle(frame,(0,0),(224,224),(0,225,0),4)
    roi=frame[0:224,0:224]
    
    
    
    cv2.imshow("roi",roi)
    cv2.imshow("Video",frame)
    
    if cv2.waitKey(25) & 0xff==ord("c"):
        cv2.imwrite("validating_data\\2\\{}.jpg".format(k),roi)
        k+=1
    if cv2.waitKey(25) & 0xff==ord("x"):
        break
    
cap.release()
cv2.destroyAllWindows()

In [14]:
# Creating model for hand or not

import cv2
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Activation



from tensorflow.keras.constraints import max_norm
from tensorflow.keras.optimizers import SGD

batch_size=64

train_datagen=ImageDataGenerator(rescale=1/225)
val_datagen=ImageDataGenerator(rescale=1/225)


In [15]:
train_generator=train_datagen.flow_from_directory("training_data",target_size=(224,224),batch_size=batch_size,class_mode="categorical")
val_generator=val_datagen.flow_from_directory("validating_data",target_size=(224,224),batch_size=batch_size,class_mode="categorical")

Found 150 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [30]:
model=Sequential()

model.add(Conv2D(32,(3,3),input_shape=(224,224,3),padding="same",activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),input_shape=(224,224,3),padding="same",activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(64,activation="relu"))

model.add(Dense(3,activation="sigmoid"))

In [31]:
sgd=SGD(learning_rate=0.01,momentum=0.9)
model.compile(loss="binary_crossentropy",optimizer=sgd,metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 56, 56, 64)        0         
                                                                 
 flatten_4 (Flatten)         (None, 200704)           

In [34]:
model.fit_generator(train_generator,train_generator.n//batch_size,epochs=1,validation_data=val_generator,validation_steps=val_generator.n//batch_size,verbose=1)

C:\Users\Dell\AppData\Local\Temp\ipykernel_11612\617599834.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,train_generator.n//batch_size,epochs=1,validation_data=val_generator,validation_steps=val_generator.n//batch_size,verbose=1)


2/2 [==============================] - 3s 3s/step - loss: 0.6256 - accuracy: 0.4767


In [35]:
from tensorflow.keras.models import save_model

In [36]:
save_model(model,"thisbetterwork.h5")